# Notebook to generate `indicators` module

In [67]:
import inspect

from pathlib import Path

from pprint import pformat

from mintalib import core

from inspect import Signature

from mintalib.samples import sample_prices
from mintalib.builder import annotate_parameter


class TEXT(str):
    def __repr__(self):
        return self

In [68]:

ROOTDIR = Path.cwd().parent
PKGDIR = ROOTDIR.joinpath("src/mintalib").resolve(strict=True)



In [69]:

prices = sample_prices()


In [70]:
import importlib.util

def new_module(name: str, code: str = None):
    spec = importlib.util.spec_from_loader(name, None)
    module = importlib.util.module_from_spec(spec)
    if code:
        exec(code, module.__dict__)
    return module

In [71]:
def core_functions():
    return list(k for k, v in vars(core).items() if k.startswith("calc_") and callable(v))

core_functions()

['calc_avgprice',
 'calc_typprice',
 'calc_wclprice',
 'calc_midprice',
 'calc_price',
 'calc_crossover',
 'calc_crossunder',
 'calc_flag',
 'calc_updown',
 'calc_sign',
 'calc_log',
 'calc_exp',
 'calc_diff',
 'calc_lag',
 'calc_min',
 'calc_max',
 'calc_sum',
 'calc_roc',
 'calc_mad',
 'calc_stdev',
 'calc_sma',
 'calc_ema',
 'calc_rma',
 'calc_wma',
 'calc_hma',
 'calc_dema',
 'calc_tema',
 'calc_ma',
 'calc_rsi',
 'calc_dmi',
 'calc_adx',
 'calc_pdi',
 'calc_mdi',
 'calc_trange',
 'calc_atr',
 'calc_natr',
 'calc_sar',
 'calc_cci',
 'calc_cmf',
 'calc_mfi',
 'calc_bop',
 'calc_bbands',
 'calc_keltner',
 'calc_ker',
 'calc_kama',
 'calc_macd',
 'calc_ppo',
 'calc_slope',
 'calc_rvalue',
 'calc_tsf',
 'calc_curve',
 'calc_stoch',
 'calc_streak',
 'calc_eval']

In [72]:
# Sample code for indicator

from mintalib.core import wrap_indicator
from mintalib.model import FuncIndicator

@wrap_indicator(core.calc_ema)
def SMA(period: int = 3):
    params = dict(locals())
    return FuncIndicator(core.calc_sma, params=params)


In [73]:
SMA(20) @ prices


date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16           NaN
1980-12-17           NaN
1980-12-18           NaN
                 ...    
2024-10-15    227.524000
2024-10-16    228.078500
2024-10-17    228.242500
2024-10-18    228.582500
2024-10-21    229.082999
Length: 11056, dtype: float64

In [74]:

PREAMBLE = '''"""
Factory functions for technical analysis indicators.

Indicator factory names are all upper case.

Indicators offer a composable interface where a calculation routine
is bound together with its calculation parameters.

An indicator object is a callable that can be applied to prices or series data.

Indicators can be chained with the `@` operator as in `ROC(1) @ SMA(20)`.

The `@` operator can also be used to apply an indicator to its parameter.

So for example `SMA(50) @ prices` can be used to compute the 50 period simple moving average on `prices`,
instead of the more verbose `SMA(50)(prices)`.
"""

# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from .core import wrap_indicator
from .model import FuncIndicator

nan = float('nan')


'''

def short_name(func):
    name = func.__name__
    if name.startswith("calc_"):
        name = name.removeprefix("calc_")
    return name


def make_indicator(func, name: str = None, preamble=False, verbose=False):
    if name is None:
        name = short_name(func).upper()
    
    fname = func.__qualname__
    module = func.__module__.rpartition(".")[2]
    qname = f"{module}.{fname}"

    signature = Signature.from_callable(func)
    parameters = list(signature.parameters.values())

    if verbose:
        print("signature", signature)
        print("parameters", parameters)

    newparams = []
    ftype = parameters[0].name

    newparams = [annotate_parameter(p) for p in parameters[1:] if p.name not in ("wrap", )]

    if ftype == "series":
        newparams.append(inspect.Parameter("item", kind=inspect.Parameter.KEYWORD_ONLY, default=None, annotation=str))

    if verbose:
        print("newparams", newparams)

    newsig = Signature(newparams)

    def argument(arg):
        return ftype if arg == "self" else f"{arg}=self.{arg}"

    code = PREAMBLE if preamble else ""
    # code = "# noinspection PyPep8Naming\n"

    code += f"@wrap_indicator({qname})\n"
    code += f"def {name}{newsig}:\n"

    kwargs = ", ".join(f"{p.name}={p.name}" for p in newparams)
    code += f"    params = dict({kwargs})\n"

    code += f"    return FuncIndicator({qname}, params=params)\n"


    return code


TEXT(make_indicator(core.calc_ema))

@wrap_indicator(core.calc_ema)
def EMA(period: int, *, adjust: bool = False, item: str = None):
    params = dict(period=period, adjust=adjust, item=item)
    return FuncIndicator(core.calc_ema, params=params)

In [75]:
def test_indicator(func, name=None, verbose=False):
    code = make_indicator(func, name=name, preamble=True, verbose=verbose)
    return new_module("mintalib.functions", code)


test_indicator(core.calc_ema)

<module 'mintalib.functions'>

In [76]:

def make_indicators(cnames=None):
    if cnames is None:
        cnames = core_functions()

    output = PREAMBLE

    output += "\n\n"
    names = []

    for cname in cnames:
        func = getattr(core, cname)
        name = short_name(func).upper()
        names.append(name)
        text = make_indicator(func, name=name)
        output += text + "\n\n"

    # output += "__all__ = [name for name in dir() if name.isupper()]\n"

    buffer = pformat(names, width=75, compact=True, indent=4)
    buffer = buffer.replace("[", " ").replace("]", "")
    output += f"__all__ = [\n{buffer}\n]\n"

    return output


code = make_indicators()

TEXT(code[:1024] +  " ...")

"""
Factory functions for technical analysis indicators.

Indicator factory names are all upper case.

Indicators offer a composable interface where a calculation routine
is bound together with its calculation parameters.

An indicator object is a callable that can be applied to prices or series data.

Indicators can be chained with the `@` operator as in `ROC(1) @ SMA(20)`.

The `@` operator can also be used to apply an indicator to its parameter.

So for example `SMA(50) @ prices` can be used to compute the 50 period simple moving average on `prices`,
instead of the more verbose `SMA(50)(prices)`.
"""

# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from .core import wrap_indicator
from .model import FuncIndicator

nan = float('nan')




@wrap_indicator(core.calc_avgprice)
def AVGPRICE():
    params = dict()
    return FuncIndicator(core.calc_avgprice, params=params)


@wrap_indicator(core.calc_typprice)
def TYPPRICE():
    params = dict()
    retu

In [77]:
code = make_indicators()

indicators = new_module("mintalib.indicators", code)

dir(indicators)

['ADX',
 'ATR',
 'AVGPRICE',
 'BBANDS',
 'BOP',
 'CCI',
 'CMF',
 'CROSSOVER',
 'CROSSUNDER',
 'CURVE',
 'DEMA',
 'DIFF',
 'DMI',
 'EMA',
 'EVAL',
 'EXP',
 'FLAG',
 'FuncIndicator',
 'HMA',
 'KAMA',
 'KELTNER',
 'KER',
 'LAG',
 'LOG',
 'MA',
 'MACD',
 'MAD',
 'MAX',
 'MDI',
 'MFI',
 'MIDPRICE',
 'MIN',
 'NATR',
 'PDI',
 'PPO',
 'PRICE',
 'RMA',
 'ROC',
 'RSI',
 'RVALUE',
 'SAR',
 'SIGN',
 'SLOPE',
 'SMA',
 'STDEV',
 'STOCH',
 'STREAK',
 'SUM',
 'TEMA',
 'TRANGE',
 'TSF',
 'TYPPRICE',
 'UPDOWN',
 'WCLPRICE',
 'WMA',
 '__all__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'core',
 'nan',
 'wrap_indicator']

In [78]:
assert callable(indicators.SMA(20))

res = indicators.SMA(20) @ prices

assert len(res) > 0


In [79]:
outfile = PKGDIR / "indicators.py"

print(f"Updating {outfile.name} ...")

outfile.write_text(code)

Updating indicators.py ...


10720

In [80]:
from mintalib import indicators

assert callable(indicators.SMA(20))

res = indicators.SMA(20) @ prices

assert len(res) > 0